<a href="https://colab.research.google.com/github/anphantt2406/Heritage-Health-Prize/blob/main/PrepareData2years.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/HHP_release3/

Mounted at /content/drive
/content/drive/My Drive/HHP_release3


# Import Library

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()

# Claims process

Combine the **Claims data of Year 1**, **Claims data of Year 2**, and **DaysInHospital in Year 2** to predicts how many days a patient will spend in hospital in Year 3.

Hence, we only select patients who has been made claims in Y2 or both Y1 and Y2 **(71435 patients)**.

In [3]:
claims_df = pd.read_csv("3_Claims.csv").query("Year == 'Y1' or Year == 'Y2'").reset_index(drop=True)
claims_df.columns = ['MemberID', 'Provider', 'Vendor', 'PCP', 'Year', 'Specialty',
       'Place', 'PayDelay', 'LOS', 'DSFS', 'PCG',
       'Charlson', 'Procedure', 'SupLOS']

claims_Y3_df = claims_df.query("Year == 'Y3'").drop(['Year'], axis=1).reset_index(drop=True)

### PayDelay

In [ ]:
claims_df.loc[claims_df['PayDelay']=='162+','PayDelay'] = 162
claims_df['PayDelay'] = claims_df['PayDelay'].astype(int)

### Length of Stay

In [ ]:
claims_df.loc[claims_df['LOS']=='1 day','LOS'] = 1
claims_df.loc[claims_df['LOS']=='2 days','LOS'] = 2
claims_df.loc[claims_df['LOS']=='3 days','LOS'] = 3
claims_df.loc[claims_df['LOS']=='4 days','LOS'] = 4
claims_df.loc[claims_df['LOS']=='5 days','LOS'] = 5
claims_df.loc[claims_df['LOS']=='6 days','LOS'] = 6
claims_df.loc[claims_df['LOS']=='1- 2 weeks','LOS'] = 11
claims_df.loc[claims_df['LOS']=='2- 4 weeks','LOS'] = 21
claims_df.loc[claims_df['LOS']=='4- 8 weeks','LOS'] = 42
claims_df.loc[claims_df['LOS']=='26+ weeks','LOS'] = 180

claims_df['LOS'].fillna(0, inplace=True)
claims_df['LOS'] = claims_df['LOS'].astype(int)

##  Process data of Unique MemberID

In [ ]:
#compute the number of provider, vendor, claim, ... for each unique member for 2 years
claims_grouped2 = pd.DataFrame(claims_df.groupby(['MemberID']).agg({
    'Provider': ['count','nunique'],
    'Specialty' : 'nunique',
    'PCG' : 'nunique',
    'Procedure' : 'nunique',
    'PayDelay' : 'sum',
    'LOS' : 'sum'
}).reset_index())
claims_grouped2.columns = ['MemberID', 'no_Claims', 'no_Providers','no_Specialties','no_PCG',\
                             'no_Procedure','sum_PayDelay','sum_LOS']

claims_grouped2

,MemberID,no_Claims,no_Providers,no_Specialties,no_PCG,no_Procedure,sum_PayDelay,sum_LOS
0,4,1,1,1,1,1,43,0
1,210,14,5,4,4,5,1028,2
2,3197,10,6,5,3,5,640,0
3,3457,1,1,1,1,1,63,0
4,3713,10,5,3,5,3,562,0
...,...,...,...,...,...,...,...,...
95502,99996214,1,1,1,1,1,19,0
95503,99997485,1,1,1,1,1,130,0
95504,99997895,14,5,4,6,4,539,0
95505,99998627,10,7,5,3,7,526,2


In [ ]:
#one-hot encoding
claims_cat = claims_df.select_dtypes(include=['object'])
cat_cols = list(claims_cat.columns)
cat_cols.remove('Year')

for col in cat_cols:
  onehot_features = pd.get_dummies(claims_cat[col])
  labels = list(onehot_features.columns) 
  for i in range(len(labels)):
    col_encode = col + "_" + labels[i]
    claims_df[col_encode] = onehot_features[onehot_features.columns[i]]
  claims_df = claims_df.drop([col], axis=1)

claims_encode = claims_df.reset_index(drop=True)
claims_encode = claims_encode.drop(['Year','Provider','Vendor','PCP','LOS','SupLOS','PayDelay'],axis =1)
claims_encode.sort_values('MemberID',ignore_index=True)

#count one-hot values base on memberID
encode_group = pd.DataFrame()
encode_group['MemberID'] = claims_grouped2['MemberID']
count_unique = list(claims_encode.columns)
count_unique.remove('MemberID')

for col in count_unique:
  df = claims_encode.groupby(['MemberID'])[col].agg('sum')
  encode_group = pd.merge(encode_group, df, on=['MemberID'])

claims_processed = pd.merge(claims_grouped2, encode_group, on=['MemberID'])

In [ ]:
claims_processed

,MemberID,no_Claims,no_Providers,no_Specialties,no_PCG,no_Procedure,sum_PayDelay,sum_LOS,Specialty_Anesthesiology,Specialty_Diagnostic Imaging,Specialty_Emergency,Specialty_General Practice,Specialty_Internal,Specialty_Laboratory,Specialty_Obstetrics and Gynecology,Specialty_Other,Specialty_Pathology,Specialty_Pediatrics,Specialty_Rehabilitation,Specialty_Surgery,Place_Ambulance,Place_Home,Place_Independent Lab,Place_Inpatient Hospital,Place_Office,Place_Other,Place_Outpatient Hospital,Place_Urgent Care,DSFS_0- 1 month,DSFS_1- 2 months,DSFS_10-11 months,DSFS_11-12 months,DSFS_2- 3 months,DSFS_3- 4 months,DSFS_4- 5 months,DSFS_5- 6 months,DSFS_6- 7 months,DSFS_7- 8 months,DSFS_8- 9 months,DSFS_9-10 months,...,PCG_MSC2a3,PCG_NEUMENT,PCG_ODaBNCA,PCG_PERINTL,PCG_PERVALV,PCG_PNCRDZ,PCG_PNEUM,PCG_PRGNCY,PCG_RENAL1,PCG_RENAL2,PCG_RENAL3,PCG_RESPR4,PCG_ROAMI,PCG_SEIZURE,PCG_SEPSIS,PCG_SKNAUT,PCG_STROKE,PCG_TRAUMA,PCG_UTI,Charlson_0,Charlson_1-2,Charlson_3-4,Charlson_5+,Procedure_ANES,Procedure_EM,Procedure_MED,Procedure_PL,Procedure_RAD,Procedure_SAS,Procedure_SCS,Procedure_SDS,Procedure_SEOA,Procedure_SGS,Procedure_SIS,Procedure_SMCD,Procedure_SMS,Procedure_SNS,Procedure_SO,Procedure_SRS,Procedure_SUS
0,4,1,1,1,1,1,43,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,210,14,5,4,4,5,1028,2,0,0,2,0,5,3,0,4,0,0,0,0,0,0,3,0,9,0,0,2,5,2,0,0,0,3,0,0,2,0,0,2,...,3,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,14,0,0,0,0,2,8,2,0,0,1,1,0,0,0,0,0,0,0,0,0
2,3197,10,6,5,3,5,640,0,0,1,2,0,1,2,0,0,0,4,0,0,0,0,2,0,6,0,0,2,2,5,0,2,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,10,0,0,0,0,6,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0
3,3457,1,1,1,1,1,63,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3713,10,5,3,5,3,562,0,0,1,0,7,0,2,0,0,0,0,0,0,0,0,2,0,8,0,0,0,1,5,0,0,0,0,0,0,0,0,4,0,...,2,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,2,10,0,0,0,0,5,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95502,99996214,1,1,1,1,1,19,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
95503,99997485,1,1,1,1,1,130,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
95504,99997895,14,5,4,6,4,539,0,0,1,0,0,4,7,0,0,0,0,0,2,0,0,7,0,7,0,0,0,2,1,0,0,1,2,2,0,0,0,3,1,...,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,14,0,0,0,0,6,0,4,1,0,3,0,0,0,0,0,0,0,0,0,0
95505,99998627,10,7,5,3,7,526,2,1,1,0,0,4,2,0,2,0,0,0,0,0,0,2,0,3,0,5,0,10,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,6,4,0,0,1,2,1,1,1,0,2,0,0,0,0,0,2,0,0,0,0


# Member of 2 Years
We reuse the Member List of Year 2 - which has been processed in *RepareDate.ipynb*

In [ ]:
member = pd.read_csv('/content/drive/My Drive/HHP_release3/new_data/memberY2.csv')

# Drug & Lab Count

In [ ]:
drug_df = pd.read_csv("DrugCount.csv")
# Replace DrugCount 7+ with 7
drug_df.loc[drug_df['DrugCount']=='7+','DrugCount'] = 7
drug_df['DrugCount'] = drug_df['DrugCount'].astype(int)
drug_df = drug_df.drop(['DSFS'], axis = 1)

lab_df = pd.read_csv("LabCount.csv")
# Replace LabCount 10+ with 10
lab_df.loc[lab_df['LabCount']=='10+', 'LabCount'] = 10
lab_df['LabCount'] = lab_df['LabCount'].astype(int)
lab_df = lab_df.drop(['DSFS'], axis = 1)

drug_2y_df = drug_df.query("Year == 'Y1' or Year == 'Y2'").drop(['Year'], axis = 1).reset_index(drop=True) 
lab_2y_df = lab_df.query("Year == 'Y1' or Year == 'Y2'").drop(['Year'], axis = 1).reset_index(drop=True) 

In [ ]:
drug_2y_df.sort_values('MemberID', ignore_index=True)
drug_group = pd.DataFrame(drug_2y_df.groupby(['MemberID'])['DrugCount'].agg('sum').reset_index())
drug_group.columns = ['MemberID','DrugSum']

drug2y = pd.DataFrame()
drug2y['MemberID'] = claims_processed['MemberID'].astype(int)
drug2y['DrugSum'] = 0
drug2y['DrugSum'] = drug2y['DrugSum'].astype(int)

for id in drug_group.MemberID:
  drug2y.drop(drug2y.index[drug2y['MemberID'] == id], inplace = True)

drug2y = drug2y.append(drug_group)
drug2y.sort_values('MemberID', ignore_index=True)

,MemberID,DrugSum
0,4,0
1,210,5
2,3197,8
3,3457,0
4,3713,17
...,...,...
95502,99996214,0
95503,99997485,0
95504,99997895,0
95505,99998627,1


In [ ]:
lab_2y_df.sort_values('MemberID', ignore_index=True)
lab_group = pd.DataFrame(lab_2y_df.groupby(['MemberID'])['LabCount'].agg('sum').reset_index())
lab_group.columns = ['MemberID','LabSum']

lab_2y = pd.DataFrame()
lab_2y['MemberID'] = claims_processed['MemberID'].astype(int)
lab_2y['LabSum'] = 0
lab_2y['LabSum'] = lab_2y['LabSum'].astype(int)

for id in lab_group.MemberID:
  lab_2y.drop(lab_2y.index[lab_2y['MemberID'] == id], inplace = True)
lab_2y = lab_2y.append(lab_group)
lab_2y.sort_values('MemberID', ignore_index=True)

,MemberID,LabSum
0,4,0
1,210,3
2,3197,2
3,3457,0
4,3713,9
...,...,...
95502,99996214,0
95503,99997485,0
95504,99997895,10
95505,99998627,5


In [ ]:
labdrug_2years = pd.merge(drug2y, lab_2y, on=['MemberID'], how='outer')
labdrug_2years = labdrug_2years.sort_values('MemberID', ignore_index=True)

In [ ]:
target = pd.read_csv('/content/drive/My Drive/HHP_release3/new_data/DaysInHos_Y2.csv')
DIHY1 = pd.read_csv('/content/drive/My Drive/HHP_release3/new_data/DaysInHos_Y1.csv')

In [ ]:
target = target.drop(['CLASS'], axis = 1)
target.rename(columns={"DaysInHospital": "TARGET"}, inplace=True)

In [ ]:
DIHY1 = DIHY1.drop(['CLASS'], axis = 1)
DIHY1.rename(columns={"DaysInHospital": "DIHY1"}, inplace=True)

# Merge and Save data

In [ ]:
claim2years = pd.merge(claims_processed, labdrug_2years, on=['MemberID'])

In [ ]:
claims = pd.DataFrame()
for id in member.MemberID:
  claims = claims.append(claims_processed.loc[claims_processed.MemberID == id])
claims.sort_values('MemberID',ignore_index=True)

,MemberID,no_Claims,no_Providers,no_Specialties,no_PCG,no_Procedure,sum_PayDelay,sum_LOS,Specialty_Anesthesiology,Specialty_Diagnostic Imaging,Specialty_Emergency,Specialty_General Practice,Specialty_Internal,Specialty_Laboratory,Specialty_Obstetrics and Gynecology,Specialty_Other,Specialty_Pathology,Specialty_Pediatrics,Specialty_Rehabilitation,Specialty_Surgery,Place_Ambulance,Place_Home,Place_Independent Lab,Place_Inpatient Hospital,Place_Office,Place_Other,Place_Outpatient Hospital,Place_Urgent Care,DSFS_0- 1 month,DSFS_1- 2 months,DSFS_10-11 months,DSFS_11-12 months,DSFS_2- 3 months,DSFS_3- 4 months,DSFS_4- 5 months,DSFS_5- 6 months,DSFS_6- 7 months,DSFS_7- 8 months,DSFS_8- 9 months,DSFS_9-10 months,...,PCG_MSC2a3,PCG_NEUMENT,PCG_ODaBNCA,PCG_PERINTL,PCG_PERVALV,PCG_PNCRDZ,PCG_PNEUM,PCG_PRGNCY,PCG_RENAL1,PCG_RENAL2,PCG_RENAL3,PCG_RESPR4,PCG_ROAMI,PCG_SEIZURE,PCG_SEPSIS,PCG_SKNAUT,PCG_STROKE,PCG_TRAUMA,PCG_UTI,Charlson_0,Charlson_1-2,Charlson_3-4,Charlson_5+,Procedure_ANES,Procedure_EM,Procedure_MED,Procedure_PL,Procedure_RAD,Procedure_SAS,Procedure_SCS,Procedure_SDS,Procedure_SEOA,Procedure_SGS,Procedure_SIS,Procedure_SMCD,Procedure_SMS,Procedure_SNS,Procedure_SO,Procedure_SRS,Procedure_SUS
0,4,1,1,1,1,1,43,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,210,14,5,4,4,5,1028,2,0,0,2,0,5,3,0,4,0,0,0,0,0,0,3,0,9,0,0,2,5,2,0,0,0,3,0,0,2,0,0,2,...,3,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,14,0,0,0,0,2,8,2,0,0,1,1,0,0,0,0,0,0,0,0,0
2,3197,10,6,5,3,5,640,0,0,1,2,0,1,2,0,0,0,4,0,0,0,0,2,0,6,0,0,2,2,5,0,2,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,10,0,0,0,0,6,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0
3,3457,1,1,1,1,1,63,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3713,10,5,3,5,3,562,0,0,1,0,7,0,2,0,0,0,0,0,0,0,0,2,0,8,0,0,0,1,5,0,0,0,0,0,0,0,0,4,0,...,2,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,2,10,0,0,0,0,5,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71430,99985996,5,2,2,2,2,316,0,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,2,1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71431,99987030,4,3,2,1,3,125,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
71432,99995391,11,2,2,4,4,533,0,0,0,0,0,6,0,0,0,0,0,0,5,0,0,0,0,11,0,0,0,2,1,0,0,4,2,0,2,0,0,0,0,...,0,5,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,11,0,0,0,0,3,6,1,1,0,0,0,0,0,0,0,0,0,0,0,0
71433,99995554,50,3,3,5,4,4480,0,0,0,0,4,37,9,0,0,0,0,0,0,0,0,9,0,41,0,0,0,11,2,0,0,3,2,1,6,7,7,4,7,...,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,40,0,0,0,50,0,0,0,0,6,35,4,0,0,5,0,0,0,0,0,0,0,0,0,0


In [ ]:
data2years = pd.merge(claims, DIHY1, on = ['MemberID'], how = 'left')
data2years['DIHY1'].fillna(-1, inplace=True)
#Patients who hadn't been made Claims in Year 1 will be marked by -1, to distinguish with who spend 0 day in hospital in Year 1. 
data2years['DIHY1'] = data2years['DIHY1'].astype(int)

In [ ]:
data2years = pd.merge(data2years, target, on = ['MemberID'])

In [ ]:
data2years

,MemberID,no_Claims,no_Providers,no_Specialties,no_PCG,no_Procedure,sum_PayDelay,sum_LOS,Specialty_Anesthesiology,Specialty_Diagnostic Imaging,Specialty_Emergency,Specialty_General Practice,Specialty_Internal,Specialty_Laboratory,Specialty_Obstetrics and Gynecology,Specialty_Other,Specialty_Pathology,Specialty_Pediatrics,Specialty_Rehabilitation,Specialty_Surgery,Place_Ambulance,Place_Home,Place_Independent Lab,Place_Inpatient Hospital,Place_Office,Place_Other,Place_Outpatient Hospital,Place_Urgent Care,DSFS_0- 1 month,DSFS_1- 2 months,DSFS_10-11 months,DSFS_11-12 months,DSFS_2- 3 months,DSFS_3- 4 months,DSFS_4- 5 months,DSFS_5- 6 months,DSFS_6- 7 months,DSFS_7- 8 months,DSFS_8- 9 months,DSFS_9-10 months,...,PCG_ODaBNCA,PCG_PERINTL,PCG_PERVALV,PCG_PNCRDZ,PCG_PNEUM,PCG_PRGNCY,PCG_RENAL1,PCG_RENAL2,PCG_RENAL3,PCG_RESPR4,PCG_ROAMI,PCG_SEIZURE,PCG_SEPSIS,PCG_SKNAUT,PCG_STROKE,PCG_TRAUMA,PCG_UTI,Charlson_0,Charlson_1-2,Charlson_3-4,Charlson_5+,Procedure_ANES,Procedure_EM,Procedure_MED,Procedure_PL,Procedure_RAD,Procedure_SAS,Procedure_SCS,Procedure_SDS,Procedure_SEOA,Procedure_SGS,Procedure_SIS,Procedure_SMCD,Procedure_SMS,Procedure_SNS,Procedure_SO,Procedure_SRS,Procedure_SUS,DIHY1,TARGET
0,4,1,1,1,1,1,43,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0
1,210,14,5,4,4,5,1028,2,0,0,2,0,5,3,0,4,0,0,0,0,0,0,3,0,9,0,0,2,5,2,0,0,0,3,0,0,2,0,0,2,...,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,14,0,0,0,0,2,8,2,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2,3197,10,6,5,3,5,640,0,0,1,2,0,1,2,0,0,0,4,0,0,0,0,2,0,6,0,0,2,2,5,0,2,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,10,0,0,0,0,6,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0
3,3457,1,1,1,1,1,63,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0
4,3713,10,5,3,5,3,562,0,0,1,0,7,0,2,0,0,0,0,0,0,0,0,2,0,8,0,0,0,1,5,0,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,2,10,0,0,0,0,5,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71430,99985996,5,2,2,2,2,316,0,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,2,1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0
71431,99987030,4,3,2,1,3,125,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0
71432,99995391,11,2,2,4,4,533,0,0,0,0,0,6,0,0,0,0,0,0,5,0,0,0,0,11,0,0,0,2,1,0,0,4,2,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,11,0,0,0,0,3,6,1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0
71433,99995554,50,3,3,5,4,4480,0,0,0,0,4,37,9,0,0,0,0,0,0,0,0,9,0,41,0,0,0,11,2,0,0,3,2,1,6,7,7,4,7,...,0,0,0,0,0,0,0,0,0,2,0,0,0,40,0,0,0,50,0,0,0,0,6,35,4,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
%cd /content/drive/My Drive/HHP_release3/new_data

/content/drive/My Drive/HHP_release3/new_data


In [ ]:
data2years.to_csv('data2years.csv', index=False)